In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import IPython
from six.moves import urllib

print(tf.__version__)

2.0.0-beta1


In [2]:
data = pd.read_csv(r'D:\MSDS-Capstone-Project\implementation\Data\00Z.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')
data.set_index('Date', inplace=True)
del data['lifted index (500-100mb)']
data.head()

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,500mb wind speed,700mb Geopotenial height,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed
Date,,,,,,,,,,,,,,,,,,,,,
1979-01-01,270.207,265.858,91.04090,-6.230320,274.500,94002.1,13.49730,74.8817,147.309,9.97695,...,28.3270,2999.96,264.499,15.1763,1479.54,269.020,10.03370,811.678,269.909,9.19478
1979-01-02,264.428,259.792,24.69000,-0.985906,268.270,94584.4,7.05870,72.3555,123.847,13.98630,...,31.5334,2981.94,257.562,17.3507,1502.43,261.811,10.99320,851.542,263.443,9.70436
1979-01-03,263.799,258.400,8.49564,-0.217627,267.284,94704.8,3.75615,68.6644,109.633,16.00460,...,28.5733,2995.40,260.379,17.2256,1505.63,262.088,10.57410,855.933,263.871,8.09015
1979-01-04,268.385,263.466,10.50020,0.622686,271.805,94787.3,7.11649,71.6237,150.811,14.36030,...,27.9834,3035.66,262.845,15.7289,1527.98,266.258,10.72980,866.666,268.853,8.11572
1979-01-05,268.484,264.415,4.29541,-0.115587,271.764,94998.6,9.66435,76.3157,209.118,11.20880,...,24.1182,3061.01,264.006,14.5905,1546.43,267.468,8.77463,883.742,268.810,7.78784


In [5]:
data.index = data.index.shift(-1, freq='d') # convert from UTC day.. 00Z is next day, but for US it is still same day. So subtract 1 day from it

In [6]:
data.head()

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,500mb wind speed,700mb Geopotenial height,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed
Date,,,,,,,,,,,,,,,,,,,,,
1978-12-31,270.207,265.858,91.04090,-6.230320,274.500,94002.1,13.49730,74.8817,147.309,9.97695,...,28.3270,2999.96,264.499,15.1763,1479.54,269.020,10.03370,811.678,269.909,9.19478
1979-01-01,264.428,259.792,24.69000,-0.985906,268.270,94584.4,7.05870,72.3555,123.847,13.98630,...,31.5334,2981.94,257.562,17.3507,1502.43,261.811,10.99320,851.542,263.443,9.70436
1979-01-02,263.799,258.400,8.49564,-0.217627,267.284,94704.8,3.75615,68.6644,109.633,16.00460,...,28.5733,2995.40,260.379,17.2256,1505.63,262.088,10.57410,855.933,263.871,8.09015
1979-01-03,268.385,263.466,10.50020,0.622686,271.805,94787.3,7.11649,71.6237,150.811,14.36030,...,27.9834,3035.66,262.845,15.7289,1527.98,266.258,10.72980,866.666,268.853,8.11572
1979-01-04,268.484,264.415,4.29541,-0.115587,271.764,94998.6,9.66435,76.3157,209.118,11.20880,...,24.1182,3061.01,264.006,14.5905,1546.43,267.468,8.77463,883.742,268.810,7.78784


In [8]:
data = data.drop(data.index[0])
data.head()

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,500mb wind speed,700mb Geopotenial height,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed
Date,,,,,,,,,,,,,,,,,,,,,
1979-01-01,264.428,259.792,24.69000,-0.985906,268.270,94584.4,7.05870,72.3555,123.847,13.98630,...,31.5334,2981.94,257.562,17.3507,1502.43,261.811,10.99320,851.542,263.443,9.70436
1979-01-02,263.799,258.400,8.49564,-0.217627,267.284,94704.8,3.75615,68.6644,109.633,16.00460,...,28.5733,2995.40,260.379,17.2256,1505.63,262.088,10.57410,855.933,263.871,8.09015
1979-01-03,268.385,263.466,10.50020,0.622686,271.805,94787.3,7.11649,71.6237,150.811,14.36030,...,27.9834,3035.66,262.845,15.7289,1527.98,266.258,10.72980,866.666,268.853,8.11572
1979-01-04,268.484,264.415,4.29541,-0.115587,271.764,94998.6,9.66435,76.3157,209.118,11.20880,...,24.1182,3061.01,264.006,14.5905,1546.43,267.468,8.77463,883.742,268.810,7.78784
1979-01-05,269.204,265.578,15.21130,-1.640400,272.908,94350.2,10.28250,78.6459,195.688,9.33308,...,24.7374,3031.82,266.624,15.5854,1500.84,269.672,9.83302,833.819,270.324,7.52107


In [9]:
tor_count = pd.read_csv(r'D:\MSDS-Capstone-Project\implementation\Data\tornadoCounts.csv', index_col=1)
tor_count = tor_count.loc[:, ~tor_count.columns.str.contains('^Unnamed')]
tor_count.index = pd.to_datetime(tor_count.index)
tor_count.tail(20)

,count
date,
2013-12-12,0
2013-12-13,0
2013-12-14,0
2013-12-15,0
2013-12-16,0
2013-12-17,0
2013-12-18,0
2013-12-19,0
2013-12-20,3


In [10]:
tor_count.rename(columns={'count':'COUNT'}, inplace=True)
tor_count.loc[tor_count.COUNT < 5, 'countBinary'] = 0.
tor_count.loc[tor_count.COUNT >= 5, 'countBinary'] = 1.
tor_count.countBinary = tor_count.countBinary.astype(int)
tor_count.tail(20)

,COUNT,countBinary
date,,
2013-12-12,0,0
2013-12-13,0,0
2013-12-14,0,0
2013-12-15,0,0
2013-12-16,0,0
2013-12-17,0,0
2013-12-18,0,0
2013-12-19,0,0
2013-12-20,3,0


In [11]:
merge = pd.merge(data,tor_count, how='inner', left_index=True, right_index=True)
merge = merge.sample(frac=1)
merge.tail(20)

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed,COUNT,countBinary
2002-08-28,301.948,288.514,947.71200,-32.788000,308.938,93870.8,30.21030,48.8959,56.3647,-2.72670,...,282.199,6.02822,1545.90,293.219,5.51566,807.410,299.007,5.33347,8,1
2010-06-19,303.488,287.781,1085.13000,-66.334300,311.893,93704.9,28.34320,45.1923,71.1603,-2.28039,...,284.408,7.31444,1532.26,294.888,6.24340,791.415,300.129,5.58950,1,0
2007-09-10,296.755,286.179,444.50600,-8.361400,302.806,93880.8,30.25430,55.2915,56.1773,1.62208,...,279.121,9.27971,1534.51,288.985,6.87247,808.441,294.131,6.33369,0,0
2011-07-19,308.145,289.410,1220.98000,-60.445400,317.958,93286.9,36.93850,38.2401,41.7794,-2.60930,...,286.588,5.87395,1505.73,298.784,5.69023,752.669,304.325,5.38549,2,0
1991-12-02,275.218,269.888,2.36139,-1.667320,279.922,93363.9,12.38560,72.2504,76.7639,9.37138,...,268.087,16.21040,1437.92,273.253,10.75130,756.620,275.656,8.17538,1,0
2000-07-13,305.105,288.548,1066.94000,-56.178200,313.605,93714.0,28.82710,41.8223,43.7862,-3.01103,...,284.236,7.18781,1542.12,296.894,4.86248,794.783,302.502,4.63201,1,0
1980-01-11,274.145,266.229,69.10660,-4.573380,278.098,93902.6,9.36963,60.1490,135.7590,13.03050,...,266.316,17.55730,1475.72,270.926,11.24670,802.227,273.657,9.40010,0,0
1997-01-06,271.135,265.718,36.39940,-2.542540,275.230,94030.7,10.97810,69.6373,181.4550,9.57823,...,264.514,11.81640,1482.86,271.047,8.74587,808.281,272.314,7.70274,0,0
1994-01-11,276.905,270.280,11.05650,-1.922520,280.174,94160.1,11.58640,68.0341,144.8800,6.65713,...,268.573,11.45840,1511.13,277.548,7.38484,820.369,278.712,5.83572,0,0
1996-04-07,286.088,274.212,6.26626,-5.660680,293.196,93421.5,11.42320,49.7553,107.0420,6.58929,...,270.440,10.26340,1454.43,280.023,5.10801,753.400,284.619,3.93054,0,0


In [12]:
merge.head()

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed,COUNT,countBinary
1988-01-01,270.049,264.886,8.24834,-1.53625,273.057,94784.5,10.0776,70.1008,137.6000,11.130900,...,263.870,12.57700,1542.46,269.673,8.15822,872.369,271.280,7.33689,0,0
1983-07-20,304.809,290.747,1438.35000,-56.10180,312.767,93783.5,34.8077,48.3317,46.4506,-3.366060,...,285.048,7.37559,1547.63,296.883,6.72205,800.150,302.224,5.89858,0,0
1984-08-24,301.137,286.984,444.99500,-22.51920,308.494,93685.9,29.9403,46.1277,54.3611,0.261409,...,282.675,4.84895,1525.14,293.305,5.55042,787.293,298.321,5.28552,0,0
2013-10-27,288.373,278.415,158.12700,-12.68920,292.065,93640.9,15.0216,54.1252,119.4480,4.901510,...,275.288,8.62861,1490.83,284.119,6.81842,779.768,288.508,6.44224,0,0
1988-06-25,303.819,286.022,647.96800,-23.59840,312.921,93749.0,33.6575,38.7005,40.6006,-0.241932,...,283.689,6.11721,1538.42,294.458,5.75215,797.011,300.113,5.48868,3,0


In [13]:
merge.to_csv(r'D:\MSDS-Capstone-Project\implementation\Data\00Z-randomized.csv')